In [ ]:
import ipywidgets as widgets
from bqplot import pyplot as bqp
import batman
import matplotlib.pyplot as plt
import numpy as np
from exoplanet_example import get_flux
from exoplanet_example import W1b
from IPython.display import display

In [ ]:
# get the Wendelstein 1 b data
x, y, dy = W1b().get_data()

# define the linear x scale for the model
x2 = np.linspace(x[0], x[-1], len(x))

# Transit Simulation


Nutzen Sie das Programm um den Planeten- und Bahnradius zu bestimmen, indem sie die beiden Parameter variieren bis die berechnete zur beobachteten Lichtkurve passt.

In [ ]:
rp_text   = widgets.Label(value='Radius des Planeten [in Sternenradiien]')
rp_slider = widgets.FloatSlider(value=0.01, min=0.01, max=1, step=0.005, description='')

a_text    = widgets.Label(value='Große Hauptachse [in Sternenradien]')
a_slider  = widgets.FloatSlider(value=5, min=0.01, max=50, step=0.1, description='')

e_text    = widgets.Label(value='Exzentrizität')
e_slider  = widgets.FloatSlider(value=W1b.e, min=0, max=0.99, step=0.01, description='')

i_text    = widgets.Label(value='Inklination [Grad]')
i_slider  = widgets.FloatSlider(value=W1b.i, min=0, max=90, step=1, description='')

w_text    = widgets.Label(value='Apsidenlinie [Grad]')
w_slider  = widgets.FloatSlider(value=0, min=0, max=360, step=1, description='')

html = widgets.HTML(value='')

button = widgets.Button(description='Berechne!')
button_adv = widgets.Button(description='Weitere Parameter')

extras = [e_text, e_slider, i_text, i_slider, w_text, w_slider]
for e in extras:
    e.layout.visibility = 'hidden'

output = widgets.Output()

with output:
    bqp.figure(4, title='Transit Daten', constrained_layout=True)
    bqp.plot(x, y)
    line = bqp.plot(x2, np.ones_like(x2), colors=['black'])
    bqp.show()
    

def update(self):
    """Draw line in plot"""
    rp  = rp_slider.value
    a   = a_slider.value
    ecc = e_slider.value
    inc = i_slider.value
    w   = w_slider.value
    line.y = get_flux(x2, rp, a, ecc, inc, w)
    
    model = np.interp(x, x2, line.y)
    
    chi2norm = ((y - model)**2 / (dy**2)).sum() / (len(y) - 5)
    
    if chi2norm < 1:
        quality = 'sehr gut'
        color = 'darkgreen'
    elif chi2norm < 2:
        quality = 'geht so'
        color = 'orange'
    else:
        quality = 'könnte besser sein!'
        color = 'red'
    
    html.value = f'<span style="font-size:x-large; color:{color}">Qualität der Lösung: {quality}</span>'
    
    if chi2norm<1:
        html.value += f'<br/ ><span style="font-size:large; color:{color}">Tatsächliche Werte: R = {W1b.rprs:.2g}, a = {W1b.ars:.2g}</span>'
    
    
def toggle(self):
    for obj in extras:
        if obj.layout.visibility is None:
            obj.layout.visibility = 'hidden'
            button_adv.description = 'Weitere Parameter'
        else:
            obj.layout.visibility = None
            button_adv.description = 'Weniger Parameter'
            e_slider.value = W1b.e
            i_slider.value = W1b.i
            w_slider.value = 0.0
    
button.on_click(update)

button_adv.on_click(toggle)

display(
    widgets.VBox([
        widgets.HBox([
            widgets.VBox([
                rp_text, rp_slider,
                a_text, a_slider,
                e_text, e_slider,
            ]),
        widgets.VBox([
            i_text, i_slider,
            w_text, w_slider,
            ]),
        ]),
        widgets.HBox([button, button_adv]),
        output,
        html,
    ])
)